## Set Up Configuration

In [2]:
import boto3

# Set custom credentials
AWS_ACCESS_KEY_ID = "test"
AWS_SECRET_ACCESS_KEY = "test"
AWS_SESSION_TOKEN = "test"  # Optional, only needed for temporary credentials

ddb = boto3.resource(
    "dynamodb",
    region_name="us-east-1",
    endpoint_url="http://localhost:4566",
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    aws_session_token=AWS_SESSION_TOKEN  # Include this line only if you have a session token
)

## list Down Tables

In [3]:
list(ddb.tables.all())

[dynamodb.Table(name='DemoTable'),
 dynamodb.Table(name='Example'),
 dynamodb.Table(name='ShowTable'),
 dynamodb.Table(name='ShowTables'),
 dynamodb.Table(name='TODAYSDEMO')]

## Define Table Schema

In [67]:
# Define the table schema
table_name = "Example"
partition_key_name = "id"
partition_key_type = "S"  # String type

# Create the table
table = ddb.create_table(
    TableName=table_name,
    KeySchema=[
        {
            "AttributeName": partition_key_name,
            "KeyType": "HASH",
        }
    ],
    AttributeDefinitions=[
        {
            "AttributeName": partition_key_name,
            "AttributeType": partition_key_type,
        }
    ],
    ProvisionedThroughput={
        "ReadCapacityUnits": 5,
        "WriteCapacityUnits": 5,
    }
)

# Wait for the table to be created
print("Creating table, please wait...")
table.wait_until_exists()
print(f"Table '{table_name}' created successfully!")

Creating table, please wait...
Table 'Example' created successfully!


## list down tables

In [4]:
list(ddb.tables.all())

[dynamodb.Table(name='DemoTable'), dynamodb.Table(name='ShowTable')]

## Insert the item

In [24]:
# Insert the item
item = {
    "id": "1",
    "name": "Shrikant Padhy",
    "age": 23,
    "email": "shrikantpadhy18@gmail.com"
}

# Insert the item into the table
table.put_item(Item=item)

{'ResponseMetadata': {'RequestId': '6a29866e-ff39-42b4-832d-e580abe01822',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'x-amzn-requestid': '6a29866e-ff39-42b4-832d-e580abe01822',
   'x-amz-crc32': '2745614147',
   'connection': 'close',
   'date': 'Thu, 07 Mar 2024 08:22:53 GMT',
   'server': 'hypercorn-h11'},
  'RetryAttempts': 0}}

## Update Item

In [25]:
# Update an item
table.update_item(
    Key={"id": "1"},
    UpdateExpression="SET age = :new_age",
    ExpressionAttributeValues={
        ":new_age": 24
    },
    ReturnValues="UPDATED_NEW"
)

print("Item updated successfully!")

Item updated successfully!


## Get Item

In [26]:
response = table.get_item(Key={"id": "1"})
#print("res=",response)
item = response["Item"]
print(item)

{'name': 'Shrikant Padhy', 'id': '1', 'age': Decimal('24'), 'email': 'shrikantpadhy18@gmail.com'}


## list of tables

In [27]:
!awslocal dynamodb list-tables

{
    "TableNames": [
        "DemoTable",
        "ShowTable"
    ]
}


## Delete Item

In [29]:
table.delete_item(
                Key={"id": "1"}
                #ConditionExpression="info.rating <= :val",
                #ExpressionAttributeValues={":val": Decimal(str(rating))},
            )

{'ResponseMetadata': {'RequestId': 'bf837dfd-40cd-4558-be5f-befb346a4cc8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'x-amzn-requestid': 'bf837dfd-40cd-4558-be5f-befb346a4cc8',
   'x-amz-crc32': '2745614147',
   'connection': 'close',
   'date': 'Thu, 07 Mar 2024 08:29:57 GMT',
   'server': 'hypercorn-h11'},
  'RetryAttempts': 0}}

## Add more items

In [31]:
items = [{
    "id": "1",
    "name": "Shrikant Padhy",
    "age": 23,
    "email": "shrikantpadhy18@gmail.com"
    },
    {
    "id": "2",
    "name": "Sachin",
    "age": 46,
    "email": "sachin@gmail.com"
    },
    {
    "id": "3",
    "name": "gaurav",
    "age": 46,
    "email": "gaurav@gmail.com"
    }]

for item in items:
    table.put_item(Item=item)
    

    
        

## list down elements 

## scan elements

In [32]:
# Scan the table to get all items
response = table.scan()
items = response["Items"]
print(items)

[{'name': 'Shrikant Padhy', 'id': '1', 'age': Decimal('23'), 'email': 'shrikantpadhy18@gmail.com'}, {'name': 'gaurav', 'id': '3', 'age': Decimal('46'), 'email': 'gaurav@gmail.com'}, {'name': 'Sachin', 'id': '2', 'age': Decimal('46'), 'email': 'sachin@gmail.com'}]


In [ ]:
## 

In [35]:
table.put_item(Item = {
    "id": "3",
    "name": "Aditya",
    "age": 96,
    "email": "gaurav@gmail.com"
})

{'ResponseMetadata': {'RequestId': 'dee908fe-c717-49a7-9277-f5ab19aefcca',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'x-amzn-requestid': 'dee908fe-c717-49a7-9277-f5ab19aefcca',
   'x-amz-crc32': '2745614147',
   'connection': 'close',
   'date': 'Thu, 07 Mar 2024 08:41:43 GMT',
   'server': 'hypercorn-h11'},
  'RetryAttempts': 0}}

## Delete items based on certain expression

In [64]:
from decimal import Decimal

table.delete_item(
    Key={"id": "1"},
    ConditionExpression="age > :val",
    ExpressionAttributeValues={":val": 20 }
)


{'ResponseMetadata': {'RequestId': '147d2b91-7021-4d0f-95bb-16cd8f5a3b1f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'x-amzn-requestid': '147d2b91-7021-4d0f-95bb-16cd8f5a3b1f',
   'x-amz-crc32': '2745614147',
   'connection': 'close',
   'date': 'Thu, 07 Mar 2024 09:20:55 GMT',
   'server': 'hypercorn-h11'},
  'RetryAttempts': 0}}

In [81]:
# Scan the table to get all items
response = table.scan()
items = response["Items"]
print(items)

[{'preferences': {'language': 'en', 'darkMode': False}, 'address': {'zipCode': '12345', 'state': 'CA', 'city': 'Anytown', 'street': '123 Main St'}, 'isActive': True, 'friends': ['abc123', 'def456', 'ghi789'], 'skills': ['Python', 'JavaScript', 'SQL'], 'createdAt': '2022-03-15T10:30:00Z', 'reviews': [{'rating': Decimal('4'), 'product': 'Widget X', 'comment': 'Great product, would recommend.'}, {'rating': Decimal('3'), 'product': 'Gadget Y', 'comment': 'Average quality, could be better.'}], 'name': 'John Doe', 'id': '1234567890', 'metrics': {'totalOrders': Decimal('25'), 'totalRevenue': Decimal('1234')}, 'interests': ['reading', 'hiking', 'photography'], 'email': 'john.doe@example.com', 'age': Decimal('35')}, {'name': 'Aditya', 'id': '2', 'age': Decimal('90'), 'email': 'gaurav@gmail.com'}]


## Insert Huge Data

In [80]:

table.put_item(Item = {
  "id": "1234567890",
  "name": "John Doe",
  "email": "john.doe@example.com",
  "age": 35,
  "isActive": True,
  "createdAt": "2022-03-15T10:30:00Z",
  "address": {
    "street": "123 Main St",
    "city": "Anytown",
    "state": "CA",
    "zipCode": "12345"
  },
  "preferences": {
    "darkMode": False,
    "language": "en"
  },
  "skills": ["Python", "JavaScript", "SQL"],
  "friends": ["abc123", "def456", "ghi789"],
  "interests": ["reading", "hiking", "photography"],
  "skills": ["Python", "JavaScript", "SQL"],
  "friends": ["abc123", "def456", "ghi789"],
  "interests": ["reading", "hiking", "photography"],
  "skills": ["Python", "JavaScript", "SQL"],
  "friends": ["abc123", "def456", "ghi789"],
  "interests": ["reading", "hiking", "photography"],
  "skills": ["Python", "JavaScript", "SQL"],
  "friends": ["abc123", "def456", "ghi789"],
  "interests": ["reading", "hiking", "photography"],

  "reviews": [
    {
      "product": "Widget X",
      "rating": 4,
      "comment": "Great product, would recommend."
    },
    {
      "product": "Gadget Y",
      "rating": 3,
      "comment": "Average quality, could be better."
    }
  ],
  "metrics": {
    "totalOrders": 25,
    "totalRevenue": 1234
  }
})

{'ResponseMetadata': {'RequestId': 'b2767907-21ec-409a-b009-7e7d61edee73',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'x-amzn-requestid': 'b2767907-21ec-409a-b009-7e7d61edee73',
   'x-amz-crc32': '2745614147',
   'connection': 'close',
   'date': 'Thu, 07 Mar 2024 11:14:35 GMT',
   'server': 'hypercorn-h11'},
  'RetryAttempts': 0}}

In [82]:
item_data = {
  "id": "1234567890",
  "name": "John Doe",
  "email": "john.doe@example.com",
  "age": 35,
  "isActive": True,
  "createdAt": "2022-03-15T10:30:00Z",
  "address": {
    "street": "123 Main St",
    "city": "Anytown",
    "state": "CA",
    "zipCode": "12345"
  },
  "preferences": {
    "darkMode": False,
    "language": "en"
  },
  "skills": ["Python", "JavaScript", "SQL"],
  "friends": ["abc123", "def456", "ghi789"],
  "interests": ["reading", "hiking", "photography"],
  "reviews": [
    {
      "product": "Widget X",
      "rating": 4,
      "comment": "Great product, would recommend."
    },
    {
      "product": "Gadget Y",
      "rating": 3,
      "comment": "Average quality, could be better."
    }
  ],
  "metrics": {
    "totalOrders": 25,
    "totalRevenue": 1234.56
  }
}
len(str(item_data).encode('utf-8'))


662

In [83]:
# transaction

In [84]:
response = table.get_item(Key={"id": "1234567890"})
#print("res=",response)
item = response["Item"]
print(item)

{'preferences': {'language': 'en', 'darkMode': False}, 'address': {'zipCode': '12345', 'state': 'CA', 'city': 'Anytown', 'street': '123 Main St'}, 'isActive': True, 'friends': ['abc123', 'def456', 'ghi789'], 'skills': ['Python', 'JavaScript', 'SQL'], 'createdAt': '2022-03-15T10:30:00Z', 'reviews': [{'rating': Decimal('4'), 'product': 'Widget X', 'comment': 'Great product, would recommend.'}, {'rating': Decimal('3'), 'product': 'Gadget Y', 'comment': 'Average quality, could be better.'}], 'name': 'John Doe', 'id': '1234567890', 'metrics': {'totalOrders': Decimal('25'), 'totalRevenue': Decimal('1234')}, 'interests': ['reading', 'hiking', 'photography'], 'email': 'john.doe@example.com', 'age': Decimal('35')}


In [97]:
r = 'A' * 410000

table.put_item(Item = {
    "id":'11',
    "name":'A'*30000
})

{'ResponseMetadata': {'RequestId': '19045e48-582c-4143-a315-d2e14734881a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'x-amzn-requestid': '19045e48-582c-4143-a315-d2e14734881a',
   'x-amz-crc32': '2745614147',
   'connection': 'close',
   'date': 'Thu, 07 Mar 2024 11:31:59 GMT',
   'server': 'hypercorn-h11'},
  'RetryAttempts': 0}}

In [89]:
r ='A' * 410000
len(r.encode('utf-8'))

410000

In [11]:
sts = ""
number = 100000
for i in range(number):
    sts+='B'
    
print(sts)



...


In [15]:
import sys
character = 'B'

table_name = 'DemoTable'
table = ddb.Table(table_name)

def check_boundary(value):
    
    try:
        sts = character *value
        table.put_item(Item = {
            "id":'11',
            "name":sts
        })
        #dp[value] = True
        return True
        
    except Exception as e:
        #dp[value] = False
        return False

if True:
    
    low = 0
    high = int(1e8)
    while(high - low > 1):
        mid = low + (high - low)//2
        value = check_boundary(mid)
        if value == True:
            low = mid + 1
        else:
            high = mid

    if check_boundary(low) == False:
        value = character*high
        memory = sys.getsizeof(value) - 49
        print(f"it got broken at {low} having memory size {memory} bytes or we can say {memory/1024} kb")
        
        # check if high - 1 is true
        print(f"The boundary value for {low - 1} is {check_boundary(low - 1)}")
        
        
    elif check_boundary(high) == False:
        value = character*high
        memory = sys.getsizeof(value) - 49
        print(f"it got broken at {high} having memory size {memory} bytes or we can say {memory/1024} kb")
        #print(f"it got broken at {low}")
        print(f"The boundary value for {high - 1} is {check_boundary(high - 1)}")
        
    else:
        print("didnt break either...")

    

it got broken at 409593 having memory size 409593 bytes or we can say 399.9931640625 kb
The boundary value for 409592 is True


In [ ]:
## List down tables

In [5]:
list(ddb.tables.all())

[dynamodb.Table(name='DemoTable'),
 dynamodb.Table(name='ShowTable'),
 dynamodb.Table(name='ShowTables'),
 dynamodb.Table(name='TODAYSDEMO')]

In [29]:
# Get the instance of different table which is already mad

# Get the table instance
table_name = 'ShowTable'
table2 = ddb.Table(table_name)
print(table2)

dynamodb.Table(name='ShowTable')


## Batch Write Operations


In [40]:
import sys

sys.getsizeof('')

50

In [39]:
sys.getsizeof('AAAAAA')

55

In [37]:
sys.getsizeof('B'*409593)/1024

400.041015625

In [41]:
sys.getsizeof("shrikant")

57

In [47]:
(('B'*409593).__sizeof__())/1024

400.041015625

In [77]:
String = '12'*2048
sys.getsizeof(String)

4145

## limitation on primary key 

In [78]:
table_name = 'DemoTable'
table = ddb.Table(table_name)
try:
    table.put_item(Item = {
        "id": String,
        "name": "Aditya",
        "age": 96,
        "email": "gaurav@gmail.com"
    })
except Exception as e:
    print(e)


An error occurred (ValidationException) when calling the PutItem operation: Hash primary key values must be under 2048 bytes, and range primary key values must be under 1024 bytes


In [19]:
import sys

table_name = 'DemoTable'
table = ddb.Table(table_name)
char = '12'
def check_boundary(value):
    
    try:
        #sts = character *value
        table.put_item(Item = {
            "id":char*value,
            "name":"primary key constraint"
        })
        #dp[value] = True
        return True
        
    except Exception as e:
        #dp[value] = False
        return False

if True:
    
    low = 0
    high = int(1e8)
    while(high - low > 1):
        mid = low + (high - low)//2
        value = check_boundary(mid)
        if value == True:
            low = mid + 1
        else:
            high = mid

    if check_boundary(low) == False:
        #value = character*high
        memory = sys.getsizeof(char*low) - 49
        print(f"it got broken at {low} having memory size {memory} bytes or we can say {memory/1024} kb")
        
        # check if high - 1 is true
        print(f"The boundary value for {low - 1} is {check_boundary(low - 1)}")
        
        
    elif check_boundary(high) == False:
        
        memory = sys.getsizeof(char*high) - 49
        print(f"it got broken at {high} having memory size {memory} bytes or we can say {memory/1024} kb")
        #print(f"it got broken at {low}")
        print(f"The boundary value for {high - 1} is {check_boundary(high - 1)}")
        
    else:
        print("didnt break either...")


it got broken at 1025 having memory size 2050 bytes or we can say 2.001953125 kb
The boundary value for 1024 is True


## delete all the entries from show tables

In [45]:
table_name = 'ShowTable'

table = ddb.Table(table_name)
table.delete_item(Key = {'id':'11'})

{'ConsumedCapacity': {'TableName': 'ShowTable', 'CapacityUnits': 1.0},
 'ResponseMetadata': {'RequestId': '6705fd6d-21ab-4a45-b199-6ecc3e2c1987',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.0',
   'content-length': '70',
   'x-amzn-requestid': '6705fd6d-21ab-4a45-b199-6ecc3e2c1987',
   'x-amz-crc32': '3839395865',
   'connection': 'close',
   'date': 'Fri, 08 Mar 2024 12:04:42 GMT',
   'server': 'hypercorn-h11'},
  'RetryAttempts': 0}}

In [65]:
demo = ddb.Table("TODAYSDEMO")

In [52]:
# create client

client = boto3.client(
    'dynamodb',
    region_name='us-east-1',
    aws_access_key_id='test',
    aws_secret_access_key='test',
    endpoint_url='http://localhost:4566',
    config=boto3.session.Config(signature_version=None)
)

## Batch Get Item

In [74]:
show_table = ddb.Table('ShowTable')
example_table = ddb.Table('Example')

keys_to_get_show_table = [
     {'id': '2'}
]

keys_to_get_example_table = [{'id':'67'}]

batch_keys = {
    show_table.name: {
        'Keys': keys_to_get
    },
    example_table.name: {
        'Keys':[{'id':'67'}]
    }
}

response = ddb.batch_get_item(RequestItems=batch_keys)
#print(response)
results = response['Responses']
#print(results)

for table_name,values in results.items():
    print(f"table name is {table_name}")
    for value in values:
        print(value,end = "\n----END----\n")



table name is ShowTable
{'name': 'Aditya', 'id': '2', 'age': Decimal('90'), 'email': 'gaurav@gmail.com'}
----END----
table name is Example
{'name': 'Siddharth Raja', 'id': '67', 'age': Decimal('23'), 'email': 'siddharthraja947@gmail.com'}
----END----


In [102]:
!awslocal dynamodb list-tables

{
    "TableNames": [
        "DemoTable",
        "Example",
        "ShowTable",
        "ShowTables",
        "TODAYSDEMO"
    ]
}


## Batch write

In [105]:
show_table = ddb.Table('ShowTable')

items_to_write = [
    {'id': "400", 'name': "Riddhi Dutta"},
    {'id':"401","name":"Soham Agarwal"},
    {'id':"402","name":"Raj Vikramaditya"}
]

# Specify the table name and items to write
requests = [{'PostRequest': {'Item': item}} for item in items_to_write]

# Perform the batch write operation
with show_table.batch_writer() as batch:
    for request in requests:
        batch.put_item(Item=request['PostRequest']['Item'])

print("Batch write operation completed successfully.")

Batch write operation completed successfully.


In [106]:
response = show_table.scan()
print(*response['Items'])

{'name': 'Riddhi Dutta', 'id': '400'} {'name': 'Satyam Shukla', 'id': '202'} {'preferences': {'language': 'en', 'darkMode': False}, 'address': {'zipCode': '12345', 'state': 'CA', 'city': 'Anytown', 'street': '123 Main St'}, 'isActive': True, 'friends': ['abc123', 'def456', 'ghi789'], 'skills': ['Python', 'JavaScript', 'SQL'], 'createdAt': '2022-03-15T10:30:00Z', 'reviews': [{'rating': Decimal('4'), 'product': 'Widget X', 'comment': 'Great product, would recommend.'}, {'rating': Decimal('3'), 'product': 'Gadget Y', 'comment': 'Average quality, could be better.'}], 'name': 'John Doe', 'id': '1234567890', 'metrics': {'totalOrders': Decimal('25'), 'totalRevenue': Decimal('1234')}, 'interests': ['reading', 'hiking', 'photography'], 'email': 'john.doe@example.com', 'age': Decimal('35')} {'name': 'Soham Agarwal', 'id': '401'} {'name': 'manthan raut', 'id': '201'} {'name': 'Raj Vikramaditya', 'id': '402'} {'name': 'prashant padhy', 'id': '200'} {'name': 'Aditya', 'id': '2', 'age': Decimal('90'

## Perform Transactions

In [7]:
from botocore.exceptions import ClientError


transactions = [
    {
        'Put': {
            'TableName': 'Example',
            'Item': {
                'id': '1',
                'name': 'John'
            }
        }
    },
    {
         'Put': {
                    'TableName': 'ShowTable',
                    'Item': {
                         "id": "130",
                         "name": "Rajendra Sharma",
                         "age": 24,
                         "email": "rajendra.sharma@gmail.com"
                }
            }
    }
]

# Execute the TransactWriteItems operation
try:
    response = ddb.meta.client.transact_write_items(TransactItems=transactions)
    print("Transaction successful.")
except Exception as e:
    print(f"Error executing transaction: {e}")

Transaction successful.


## Example of failed transaction

In [26]:
from botocore.exceptions import ClientError


transactions = [
    {
        'Put': {
            'TableName': 'Example',
            'Item': {
                'id': '1',
                'name': 'B'*409599
            }
        }
    },
    {
         'Put': {
                    'TableName': 'ShowTable',
                    'Item': {
                         "id": "130",
                         "name": "Rajendra Sharma",
                         "age": 24,
                         "email": "rajendra.sharma@gmail.com"
                }
            }
    }
]

# Execute the TransactWriteItems operation
try:
    response = ddb.meta.client.transact_write_items(TransactItems=transactions)
    print("Transaction successful.")
except Exception as e:
    print(f"Error executing transaction: {e}")

Error executing transaction: An error occurred (ValidationException) when calling the TransactWriteItems operation: Item size has exceeded the maximum allowed size


In [13]:
import sys
sys.getsizeof(('B'*409593))/1024

400.041015625

In [17]:
example_table = ddb.Table('ShowTable')

In [22]:
example_table.delete_item(Key = {'id':'1302222'})

{'ConsumedCapacity': {'TableName': 'ShowTable', 'CapacityUnits': 1.0},
 'ResponseMetadata': {'RequestId': '477a90ab-81d3-48f1-9212-491b9e5bf98c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.0',
   'content-length': '70',
   'x-amzn-requestid': '477a90ab-81d3-48f1-9212-491b9e5bf98c',
   'x-amz-crc32': '3839395865',
   'connection': 'close',
   'date': 'Tue, 12 Mar 2024 08:42:41 GMT',
   'server': 'hypercorn-h11'},
  'RetryAttempts': 0}}

In [34]:
print(example_table.scan().items())

dict_items([('Items', [{'name': 'Riddhi Dutta', 'id': '400'}, {'name': 'Satyam Shukla', 'id': '202'}, {'preferences': {'language': 'en', 'darkMode': False}, 'address': {'zipCode': '12345', 'state': 'CA', 'city': 'Anytown', 'street': '123 Main St'}, 'isActive': True, 'friends': ['abc123', 'def456', 'ghi789'], 'skills': ['Python', 'JavaScript', 'SQL'], 'createdAt': '2022-03-15T10:30:00Z', 'reviews': [{'rating': Decimal('4'), 'product': 'Widget X', 'comment': 'Great product, would recommend.'}, {'rating': Decimal('3'), 'product': 'Gadget Y', 'comment': 'Average quality, could be better.'}], 'name': 'John Doe', 'id': '1234567890', 'metrics': {'totalOrders': Decimal('25'), 'totalRevenue': Decimal('1234')}, 'interests': ['reading', 'hiking', 'photography'], 'email': 'john.doe@example.com', 'age': Decimal('35')}, {'name': 'Soham Agarwal', 'id': '401'}, {'name': 'manthan raut', 'id': '201'}, {'name': 'Raj Vikramaditya', 'id': '402'}, {'name': 'prashant padhy', 'id': '200'}, {'name': 'Aditya', 

## LSI

## GSI